In [0]:
from pyspark.sql.functions import current_timestamp

In [0]:
def add_ingestion_date(input_df):
    return input_df.withColumn('ingestion_date', current_timestamp())

In [0]:
def upsert_to_delta_table(database, table, source_container, source_df, upsert_condition, partition_column):

    spark.conf.set('spark.databricks.optimizer.dynamicPartitions', 'true')

    from delta.tables import DeltaTable
    if spark._jsparkSession.catalog().tableExists(database, table):
        delta_table = DeltaTable.forPath(spark, f'{source_container}/{table}')
        delta_table.alias('target')\
            .merge(
                source_df.alias('source'),
                upsert_condition
            )\
            .whenMatchedUpdateAll()\
            .whenNotMatchedInsertAll()\
            .execute()
    else:
        source_df.write\
            .format('delta')\
            .mode('overwrite')\
            .partitionBy(partition_column)\
            .saveAsTable(f'{database}.{table}')